In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def read_ship_loading_dat(filepath):
    tanks = []
    percentages = []

    with open(filepath, 'r') as file:
        lines = file.readlines()

    i = 0
    while i < len(lines):
        tank_name = lines[i].strip()
        if tank_name == '$':  # End of file indicator
            break
        percentage = lines[i+1].strip()
        tanks.append(tank_name)
        percentages.append(float(percentage))  # convert to float if needed
        i += 2  # Move to the next tank block

    df = pd.DataFrame({
        'Tank Name': tanks,
        'Percentage Loading': percentages
    })

    return df

In [ ]:
filename='/content/Loading_basic.dat'
load=read_ship_loading_dat(filename)
print(load.head())

    Tank Name  Percentage Loading
0  NO1CARGO.P                97.9
1  NO1CARGO.S                98.0
2  NO3CARGO.P                98.4
3  NO3CARGO.S                98.3
4  NO5CARGO.P                98.2


In [ ]:
def get_tank_properties(tank_name, folder_path):
    file_path = f"{folder_path}/{tank_name}.txt"
    with open(file_path, 'r') as f:
        lines = f.readlines()

    density = float(lines[0].strip())
    permeability = float(lines[1].strip())

    # The data lines start from line 4 (index 3)
    data_lines = lines[3:]
    last_line = data_lines[-1].strip().split(',')
    max_volume = float(last_line[1])  # second column is volume

    return density, permeability, max_volume

def addto_dataframe(df, txt_folder):
    densities = []
    permeabilities = []
    max_volumes = []

    for tank in df['Tank Name']:
        density, permeability, max_volume = get_tank_properties(tank, txt_folder)
        densities.append(density)
        permeabilities.append(permeability)
        max_volumes.append(max_volume)

    df['Density'] = densities
    df['Permeability'] = permeabilities
    df['Max Volume'] = max_volumes
    df['Volume Filled'] = (df['Percentage Loading'] / 100) * df['Max Volume']
    df['Weight Filled'] = df['Volume Filled'] * df['Density']

    return df

In [ ]:
txt_folder='/content/drive/MyDrive/Structural Lab/TankSoundings'
load=addto_dataframe(load,txt_folder)
print(load.head())

    Tank Name  Percentage Loading  Density  Permeability   Max Volume  \
0  NO1CARGO.P                97.9    1.025         0.985  2136.981460   
1  NO1CARGO.S                98.0    1.025         0.985  2136.981460   
2  NO3CARGO.P                98.4    1.025         0.985  2592.270366   
3  NO3CARGO.S                98.3    1.025         0.985  2603.932683   
4  NO5CARGO.P                98.2    1.025         0.985  1401.283106   

   Volume Filled  Weight Filled  
0    2092.104849    2144.407470  
1    2094.241830    2146.597876  
2    2550.794040    2614.563891  
3    2559.665827    2623.657473  
4    1376.060010    1410.461510  


In [ ]:
def interpolate_LCG(tank_name, volume_filled, folder_path):
    file_path = f"{folder_path}/{tank_name}.txt"
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # The data starts from line 4 (index 3)
    data_lines = lines[3:]

    # Extract volume (2nd column) and LCG (4th column)
    volumes = []
    lcgs = []

    for line in data_lines:
        values = line.strip().split(',')
        volumes.append(float(values[1]))  # Volume column
        lcgs.append(float(values[3]))  # LCG column

    # Use numpy interpolation to estimate LCG for the given volume_filled
    interpolated_LCG = np.interp(volume_filled, volumes, lcgs)

    return interpolated_LCG

def add_LCG_to_dataframe(df, txt_folder):
    lcgs = []

    for index, row in df.iterrows():
        lcg = interpolate_LCG(row['Tank Name'], row['Volume Filled'], txt_folder)
        lcgs.append(lcg)

    df['Interpolated LCG'] = lcgs
    return df

In [ ]:
load=add_LCG_to_dataframe(load,txt_folder)
load.to_excel('tanks.xlsx')
print(load.head())

    Tank Name  Percentage Loading  Density  Permeability   Max Volume  \
0  NO1CARGO.P                97.9    1.025         0.985  2136.981460   
1  NO1CARGO.S                98.0    1.025         0.985  2136.981460   
2  NO3CARGO.P                98.4    1.025         0.985  2592.270366   
3  NO3CARGO.S                98.3    1.025         0.985  2603.932683   
4  NO5CARGO.P                98.2    1.025         0.985  1401.283106   

   Volume Filled  Weight Filled  Interpolated LCG  
0    2092.104849    2144.407470        137.338535  
1    2094.241830    2146.597876        137.338236  
2    2550.794040    2614.563891        104.741269  
3    2559.665827    2623.657473        104.721092  
4    1376.060010    1410.461510         73.197522  


In [ ]:
df=pd.DataFrame(
    {
       'Component':['Lightship', 'Crew and effects', 'Store'],
       'Weight':[8070, 46, 4.2],
       'LCG':[68.49, 18, 155]
    }
)

In [ ]:
total_weight=df['Weight'].sum()+load['Weight Filled'].sum()
total_LCG=(df['LCG']*df['Weight']).sum()+(load['Interpolated LCG']*load['Weight Filled']).sum()
total_LCG/=total_weight
print(total_weight)
print(total_LCG)

28263.81653510674
81.3080508270182


In [ ]:
hydrostatics_df=pd.read_excel('/content/hydrostatics.xlsx')
area_df=pd.read_excel('/content/Sectional_Area.xlsx')

In [ ]:
def get_drafts(trim, lcf, lbp, T):
  del_t_aft=(lcf/lbp)*trim
  del_t_fwd=trim-del_t_aft

  T_aft=T-del_t_aft
  T_fwd=T+del_t_fwd

  T_mean=(T_aft+T_fwd)/2
  return T_aft, T_fwd, T_mean



def get_lcf(T_mean, hydrostatics_df):
  lcf=np.interp(T_mean,hydrostatics_df['z'],hydrostatics_df['LCF'])
  return lcf



def get_sectional_area(area_df, df):
    result = []
    z_values = area_df.iloc[:,0].to_numpy()
    for _, row in df.iterrows():
        x = row['x']
        z = row['t']

        area_values = area_df[x].to_numpy()     # Sectional Area curve at x
        interpolated_area = np.interp(z, z_values, area_values) # Passing z, area arrays to interpolate

        result.append({'x': x, 'z': z, 'area': interpolated_area})

    return pd.DataFrame(result)



def get_buoyancy(area_df, T_aft, T_fwd):

  x_values=area_df.columns[1:].astype(float)
  x0=x_values[0]
  xl=x_values[-1]

  t=[]
  for i in range(len(x_values)):
    t.append(T_aft+((T_fwd-T_aft)/(xl-x0))*(x_values[i]-x0))

  df=pd.DataFrame({'x':x_values,'t':t})
  result = get_sectional_area(area_df, df)

  underwater_volume=np.trapz(result['area'],result['x'])
  buoyancy=1.025*underwater_volume
  lcb=np.trapz(result['area']*result['x'],result['x'])/underwater_volume

  return buoyancy, lcb, result

In [ ]:
W=28263.81653510674
lcg=81.3080508270182
lbp=160.8

In [ ]:
T=np.interp(W,hydrostatics_df['Weight'],hydrostatics_df['z'])
lcb=np.interp(T,hydrostatics_df['z'],hydrostatics_df['LCB'])
mctc=np.interp(T,hydrostatics_df['z'],hydrostatics_df['MTC'])
lcf=np.interp(T,hydrostatics_df['z'],hydrostatics_df['LCF'])

print(T)
print(lcf)

8.200014656728767
78.99792662398329


In [ ]:
trimming_moment=(lcg-lcb)*W
trim=trimming_moment/(mctc*100)  # Initial trim

del_t_aft=(lcf/lbp)*trim
del_t_fwd=trim-del_t_aft
print(trim)

-1.4529949102064317


In [ ]:
T_aft=T-del_t_aft
T_fwd=T+del_t_fwd

print(T_aft)
print(T_fwd)

8.913842923529195
7.460848013322764


In [ ]:
x_values=area_df.columns[1:].astype(float)

x0=x_values[0]
xl=x_values[-1]

t=[]
for i in range(len(x_values)):
  t.append(T_aft+((T_fwd-T_aft)/(xl-x0))*(x_values[i]-x0))

df=pd.DataFrame({'x':x_values,'t':t})
print(df)

              x         t
0     -3.600907  8.913843
1     -3.599993  8.913835
2     -3.000146  8.908630
3     -1.800149  8.898217
4     -1.199998  8.893009
..          ...       ...
101  163.814150  7.461075
102  163.820856  7.461017
103  163.827257  7.460962
104  163.833962  7.460904
105  163.840363  7.460848

[106 rows x 2 columns]


In [ ]:
result = get_sectional_area(area_df, df)
print(result)

              x         z      area
0     -3.600907  8.913843  0.000000
1     -3.599993  8.913835  0.000000
2     -3.000146  8.908630  0.082955
3     -1.800149  8.898217  0.923721
4     -1.199998  8.893009  1.812167
..          ...       ...       ...
101  163.814150  7.461075  0.000000
102  163.820856  7.461017  0.000000
103  163.827257  7.460962  0.000000
104  163.833962  7.460904  0.000000
105  163.840363  7.460848  0.000000

[106 rows x 3 columns]


In [ ]:
underwater_volume=np.trapz(result['area'],result['x'])
print(underwater_volume)

27580.50083291595


<ipython-input-18-ce55708c8225>:1: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  underwater_volume=np.trapz(result['area'],result['x'])


In [ ]:
buoyancy=1.025*underwater_volume
print(buoyancy)

28270.013353738846


In [ ]:
lcb=np.trapz(result['area']*result['x'],result['x'])/underwater_volume
print(lcb)

81.2632925659618


<ipython-input-20-32d078c9d2d5>:1: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  lcb=np.trapz(result['area']*result['x'],result['x'])/underwater_volume


In [ ]:
i=1
trim=-1.4529949102064317
W=28263.81653510674
lcg=81.3080508270182
lcb=81.2632925659618
lbp=160.8
T=8.200014656728767
lcf=78.99792662398329

while(abs(lcg-lcb)/lcg>0.0001):
  if(lcb>lcg):
    trim-=0.01
  else:
    trim+=0.01
  T_aft, T_fwd, T_mean=get_drafts(trim, lcf, lbp, T)
  lcf=get_lcf(T_mean, hydrostatics_df)
  buoyancy, lcb, res=get_buoyancy(area_df, T_aft, T_fwd)
  print('Iteration', i,':')
  print('T_mean=', T_mean)
  print('T_aft=', T_aft)
  print('T_fwd=', T_fwd)
  print('Trim=', trim)
  print('Weight=', W)
  print('Buoyancy=', buoyancy)
  print('LCG=', lcg)
  print('LCB=', lcb)
  print('\n\n')
  i+=1
  T=T_mean

Iteration 1 :
T_mean= 8.187432662043891
T_aft= 8.908930117147106
T_fwd= 7.465935206940675
Trim= -1.4429949102064317
Weight= 28263.81653510674
Buoyancy= 28269.841555269155
LCG= 81.3080508270182
LCB= 81.27841129751329



Iteration 2 :
T_mean= 8.175127073478286
T_aft= 8.891624528581502
T_fwd= 7.45862961837507
Trim= -1.4329949102064317
Weight= 28263.81653510674
Buoyancy= 28221.278499218446
LCG= 81.3080508270182
LCB= 81.2992088261028



Iteration 3 :
T_mean= 8.163091134307166
T_aft= 8.874588589410381
T_fwd= 7.45159367920395
Trim= -1.4229949102064317
Weight= 28263.81653510674
Buoyancy= 28173.78902907063
LCG= 81.3080508270182
LCB= 81.31989633809305



Iteration 4 :
T_mean= 8.151151625699038
T_aft= 8.867649080802254
T_fwd= 7.434654170595822
Trim= -1.4329949102064317
Weight= 28263.81653510674
Buoyancy= 28127.6750838058
LCG= 81.3080508270182
LCB= 81.3102088745479





<ipython-input-11-2cfb0c20e87e>:48: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  underwater_volume=np.trapz(result['area'],result['x'])
<ipython-input-11-2cfb0c20e87e>:50: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  lcb=np.trapz(result['area']*result['x'],result['x'])/underwater_volume
<ipython-input-11-2cfb0c20e87e>:48: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  underwater_volume=np.trapz(result['area'],result['x'])
<ipython-input-11-2cfb0c20e87e>:50: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  lcb=np.trapz(result['area']*result['x'],result['x'])/underwater_volume
<ipython-input-11-2cfb0c20e87e>:48: DeprecationWarni

In [ ]:
trim=-1.4329949102064317
W=28263.81653510674
B=28127.6750838058
lcg=81.3080508270182
lcb=81.3102088745479
T_mean=8.151151625699038
T_aft=8.867649080802254
T_fwd=7.434654170595822
i=1

while((abs(W-buoyancy)/W)>0.001):
#while(i<15):
  if(W<B):
    T_mean-=0.01
    T_aft-=0.01
    T_fwd-=0.01
  else:
    T_mean+=0.01
    T_aft+=0.01
    T_fwd+=0.01

  buoyancy, lcb, res=get_buoyancy(area_df, T_aft, T_fwd)
  print('Iteration', i,':')
  print('T_mean=', T_mean)
  print('T_aft=', T_aft)
  print('T_fwd=', T_fwd)
  print('Weight=', W)
  print('Buoyancy=', buoyancy)
  print('LCG=', lcg)
  print('LCB=', lcb)
  print('\n\n')
  i+=1


Iteration 1 :
T_mean= 8.161151625699038
T_aft= 8.877649080802254
T_fwd= 7.444654170595822
Weight= 28263.81653510674
Buoyancy= 28166.71230502433
LCG= 81.3080508270182
LCB= 81.30562229392932



Iteration 2 :
T_mean= 8.171151625699038
T_aft= 8.887649080802253
T_fwd= 7.454654170595822
Weight= 28263.81653510674
Buoyancy= 28205.755401750557
LCG= 81.3080508270182
LCB= 81.3010339251092



Iteration 3 :
T_mean= 8.181151625699037
T_aft= 8.897649080802253
T_fwd= 7.464654170595821
Weight= 28263.81653510674
Buoyancy= 28244.803092679966
LCG= 81.3080508270182
LCB= 81.29644580167859





<ipython-input-11-2cfb0c20e87e>:48: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  underwater_volume=np.trapz(result['area'],result['x'])
<ipython-input-11-2cfb0c20e87e>:50: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  lcb=np.trapz(result['area']*result['x'],result['x'])/underwater_volume
<ipython-input-11-2cfb0c20e87e>:48: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  underwater_volume=np.trapz(result['area'],result['x'])
<ipython-input-11-2cfb0c20e87e>:50: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  lcb=np.trapz(result['area']*result['x'],result['x'])/underwater_volume
<ipython-input-11-2cfb0c20e87e>:48: DeprecationWarni

In [ ]:
T_mean= 8.181151625699037
T_aft= 8.897649080802253
T_fwd= 7.464654170595821
Weight= 28263.81653510674
Buoyancy= 28244.803092679966
LCG= 81.3080508270182
LCB= 81.29644580167859

trim=T_fwd-T_aft
print(trim)

-1.4329949102064319
